Task0

In [2]:
from jedi.api.refactoring import inline
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn

You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Task1

In [3]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./mammographic_masses_data.csv"))
print(frame)

      BA   Age  Shape  Margin  Density  Severity
0    5.0  67.0    3.0     5.0      3.0         1
1    4.0  43.0    1.0     1.0      NaN         1
2    5.0  58.0    4.0     5.0      3.0         1
3    4.0  28.0    1.0     1.0      3.0         0
4    5.0  74.0    1.0     5.0      NaN         1
..   ...   ...    ...     ...      ...       ...
956  4.0  47.0    2.0     1.0      3.0         0
957  4.0  56.0    4.0     5.0      3.0         1
958  4.0  64.0    4.0     5.0      3.0         0
959  5.0  66.0    4.0     5.0      3.0         1
960  4.0  62.0    3.0     3.0      3.0         0

[961 rows x 6 columns]


In [4]:
import plotly.express as px

# 2.1
# Main functionality of pandas is reading csv files using (pd.read_csv()) 
# and converting them to a dataframe(pd.DataFrame()). to Further manipulate and visualize the data using pandas or other libraries like plotly.
# For a general look at what is inside the dataframe, pandas.DataFrame.info() works well, but for summary statistics one should use pandas.DataFrame.describe().
print(frame.describe())

# 2.2
# Makes a histogram of the Margin values of the rows in the database where Severity = 1
fig = px.histogram(frame.loc[frame.Severity == 1].Margin, x="Margin", title="Histogram of Margin with Severity = 1")
fig.show()

#2.3
# A pie-chart showing how many cases of severity = 1 were found per shape category
fig = px.pie(frame, values='Severity', names='Shape', title='Severity per Shape')
fig.show()

# A 2D heatmap with shape and severity on the axis, and the warmth depending on the frequency, accompanied by two individual histograms of the axes
fig = px.density_heatmap(frame, x="Shape", y="Severity", marginal_x="histogram", marginal_y="histogram", title="Density Heatmap of Shape and Severity")
fig.show()

# A scatter plot with on the axes the BA and Age values, and the color depending on the severity
fig = px.scatter(frame, x="BA", y="Age", color="Severity", title="Scatter Plot of BA and Age")
fig.show()

               BA         Age       Shape      Margin     Density    Severity
count  959.000000  956.000000  930.000000  913.000000  885.000000  961.000000
mean     4.300313   55.487448    2.721505    2.796276    2.910734    0.463059
std      0.683469   14.480131    1.242792    1.566546    0.380444    0.498893
min      0.000000   18.000000    1.000000    1.000000    1.000000    0.000000
25%      4.000000   45.000000    2.000000    1.000000    3.000000    0.000000
50%      4.000000   57.000000    3.000000    3.000000    3.000000    0.000000
75%      5.000000   66.000000    4.000000    4.000000    3.000000    1.000000
max      6.000000   96.000000    4.000000    5.000000    4.000000    1.000000


Task3

In [6]:
cleaned_df = frame.copy()

# 3.1
# clean the set by removing the rows containing missing data
cleaned_df = cleaned_df.dropna()

# making an histogram of both before and after removing incomplete rows
import plotly.graph_objects as go

fig = go.Figure()
fig = fig.add_trace(go.Histogram(x = frame.loc[frame.Severity == 1].Margin, name = "Original Set"))
fig = fig.add_trace(go.Histogram(x = cleaned_df.loc[cleaned_df.Severity == 1].Margin, name = "Cleaned Set"))
fig.update_layout(legend_title_text = "Dataset", title = "Histogram of Margin with Severity = 1")
fig.update_xaxes(title_text="Margin")
fig.update_yaxes(title_text="Count")
fig.show()

# 3.2
max =  -100000000000
min = 1000000000
normalized = cleaned_df.copy()

for i in cleaned_df.Age:
    if i > max:
        max = i
    if i < min:
        min = i

for i,j in enumerate(normalized.Age):
    normalized.loc[i, "Age"] = 1/(max-min) * j

print(normalized)

      BA       Age  Shape  Margin  Density  Severity
0    5.0  0.858974    3.0     5.0      3.0       1.0
2    5.0  0.358974    4.0     5.0      3.0       1.0
3    4.0  0.730769    1.0     1.0      3.0       0.0
8    5.0  0.692308    1.0     5.0      3.0       1.0
10   5.0  0.756410    1.0     4.0      3.0       1.0
..   ...       ...    ...     ...      ...       ...
745  NaN  0.794872    NaN     NaN      NaN       NaN
752  NaN  0.666667    NaN     NaN      NaN       NaN
778  NaN  0.846154    NaN     NaN      NaN       NaN
819  NaN  0.910256    NaN     NaN      NaN       NaN
824  NaN  0.666667    NaN     NaN      NaN       NaN

[959 rows x 6 columns]


Task4

In [27]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
# 4.1
# split copy to input=everything but severity and output=severity
copy_41 = cleaned_df.copy()
output = copy_41["Severity"]
input = copy_41.drop("Severity", axis=1)

# selectfrommodel part
model = SelectFromModel(estimator=LogisticRegression()) 
model.fit_transform(input, output)
selected_features = input.columns[model.get_support()]
print("Selected features:", selected_features.tolist())
feature_importances = pd.DataFrame(model.estimator_.coef_, columns=model.feature_names_in_.tolist()).transpose()
print(feature_importances)
lr_set = copy_41[selected_features.tolist() + ["Severity"]]
features = model.feature_names_in_
# Plot histograms
fig = px.bar(feature_importances, title='Feature Importance')
fig.update_layout(showlegend=False)
fig.show()

# 4.2 
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
components = pca.fit_transform(input)
print(components)
df_pca = pd.DataFrame(components, columns=["PC1", "PC2"])
df_pca["target"] = output

fig1 = px.scatter(
    df_pca, x="PC1", y="PC2", color="target",
    title="Data Projected onto First 2 PCs"
)
fig1.show()

# 4.3 
from sklearn.decomposition import TruncatedSVD
X = input.copy()
svd = TruncatedSVD(n_components=2, n_iter=9, random_state=20)
reduced = svd.fit_transform(X)

df_trunc = pd.DataFrame(reduced, columns=["Component 1", "Component 2"])
df_trunc["target"] = output

fig1 = px.scatter(
    df_trunc, x="Component 1", y="Component 2", color=df_trunc["target"],
    title="Data Projected onto First 2 Components"
)
fig1.show()

Selected features: ['BA']
                0
BA       1.563562
Age      0.050652
Shape    0.473188
Margin   0.334709
Density -0.042070


[[ 11.31599482  -1.32061038]
 [  2.36374711  -2.42483299]
 [-27.88074989   0.92984549]
 ...
 [  8.33834783  -1.9225588 ]
 [ 10.35026533  -1.96339801]
 [  6.21812803   0.13395917]]
